In [1]:
from camelup.utilities import *
from camelup.treesearch import * 
from camelup import camelup, config
from copy import deepcopy
from time import time
import numpy as np
import inspect
import ast
from io import StringIO
import re

In [2]:
game = camelup.Game(3)

camel_dict = {
    "red": {"height": 1, "space": 1, "need_roll": True},
    "blue": {"height": 2, "space": 1, "need_roll": True},
    "green": {"height": 1, "space": 2, "need_roll": True},
}
game.camel_dict = camel_dict

game.play_winner_card("blue")
game.play_loser_card("red")
game.play_bet_tile("blue")

game.state = 2
game.play_bet_tile("blue")
game.play_winner_card("green")

In [6]:
def parse_dump(string):
    """Gets information from the AST dump and return the AST object and the id

    Parameters
    ----------
    string : str
        Tree to extract the id for

    Returns
    -------
    str
        A string with the object and name info

    """
    re1 = "id='(.*?)',"
    return string.split('(')[0] + ' - ' + re.search(re1, string).group(1)


def create_benchmark_func(tree, performance):
    """Creating a benchmark function named benchmark_`func`

    Parameters
    ----------
    tree : str
        AST for function to create benchmark function for
    performance : dict
        Dictionary to capture function

    Returns
    -------
    str :
        Tree of the benchmark function

    """
    t = 0
    new_first_layer = list()
    for node in tree.body[0].body:
        if node.__class__ != ast.Return:
            node_dump = ast.dump(node)
            parsed_dump = parse_dump(node_dump)
            performance[parsed_dump] = list()
            start_time = ast.parse(f"t{t} = time()").body[0]
            t+=1
            end_time = ast.parse(f"t{t} = time()").body[0]
            dif = ast.parse(f"performance['{parsed_dump}'].append(t{t}-t{t-1})").body[0]
            new_first_layer.extend([start_time, node, end_time, dif])
            t+=1
        else:
            new_first_layer.extend([node])
    tree.body[0].body = new_first_layer
    tree.body[0].name = f'benchmark_{tree.body[0].name}'
    return tree


In [7]:
func_str = inspect.getsource(calc_utility)
tree = ast.parse(StringIO(func_str).read())
global performance
performance = dict()
exec(compile(create_benchmark_func(tree, performance), filename="<ast>", mode="exec"))
print(ast.dump(tree))

Module(body=[FunctionDef(name='benchmark_calc_utility', args=arguments(args=[arg(arg='game', annotation=None), arg(arg='iter', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Assign(targets=[Name(id='t0', ctx=Store())], value=Call(func=Name(id='time', ctx=Load()), args=[], keywords=[])), Assign(targets=[Name(id='coins', ctx=Store())], value=Call(func=Name(id='coins_to_numpy', ctx=Load()), args=[Name(id='game', ctx=Load())], keywords=[])), Assign(targets=[Name(id='t1', ctx=Store())], value=Call(func=Name(id='time', ctx=Load()), args=[], keywords=[])), Expr(value=Call(func=Attribute(value=Subscript(value=Name(id='performance', ctx=Load()), slice=Index(value=Str(s='Assign - coins')), ctx=Load()), attr='append', ctx=Load()), args=[BinOp(left=Name(id='t1', ctx=Load()), op=Sub(), right=Name(id='t0', ctx=Load()))], keywords=[])), Assign(targets=[Name(id='t2', ctx=Store())], value=Call(func=Name(id='time', ctx=Load()), args=[], keywords=[])), Assi

In [8]:
%%time
for i in range(100):
    benchmark_calc_utility(deepcopy(game),100)

CPU times: user 17.5 s, sys: 263 ms, total: 17.8 s
Wall time: 18.6 s


In [9]:
for key, val in performance.items():
    print(f'{key} = {np.sum(val)}')

Assign - coins = 0.0016665458679199219
Assign - turn_prob_first = 1.127647876739502
Assign - game_prob_first = 3.2003419399261475
Assign - winner_bets = 0.0018310546875
Assign - bet_tiles = 0.0018684864044189453
Assign - bets = 2.9970686435699463
Assign - bets_groupby = 0.3757307529449463
Assign - final = 5.141238212585449
Assign - game_first = 1.3687818050384521
Assign - game_last = 1.3058347702026367
Assign - game_winner_other = 1.1113948822021484
Assign - game_loser_other = 1.0552432537078857
Expr - calc_exp_value = 0.7527694702148438
Expr - final = 0.08347082138061523
